# Example: Reading a power grid model from the extended 3DCityDB
This example shows how to read a power grid model to the extended 3DCityDB, using the `dblayer` package. Furthermore, it shows how to use the retrieved *topographical* and *topological* data to visualize the power grid model.

Basic knowledge about [CityGML](http://www.citygmlwiki.org) and the [Utility Network ADE](https://en.wiki.utilitynetworks.sig3d.org) are required for this notebook. A nice overview of the of the basic concepts of the Utility Network ADE can be found [here](https://en.wiki.utilitynetworks.sig3d.org/images/upload/20160913_-_Kolbe_-_Modeling_3D_Utility_Networks_%2B_UtilityNetwork_ADE_Core_Model.pdf).

**Note**: In order to have the 3DCityDB populated with the data required for this example, run notebook *PowerGridModelWriteDB*.

In [ ]:
from dblayer import *
from dblayer.func.func_postgis_geom import *
from sqlalchemy import or_

## Connecting to the database
Define connection parameters for 3DCityDB instance.

In [ ]:
connect = PostgreSQLConnectionInfo(
    user = 'postgres',
    pwd = 'postgres',
    host = 'localhost',
    port = '5432',
    dbname = 'testdb'
    )

Connect to database using an instance of class `DBAccess`.

In [ ]:
access = DBAccess()
access.connect_to_citydb(connect)

## Reading data from the database
Package `dblayer` uses an object-oriented apporach for representing and retrieving information from the 3DCityDB. This means that every object defined in the database schema (e.g., `Building`, `EnergyResource`, `Cable`) is mapped to a class, whose variables correspond to the object attributes.

Therefore, before actually reading the data, the classes have to be mapped. This can be done automatically, in which case the mapping between objects and classes uses the standard `citydb` scheme. However, retrieving the mappting from this representation has some drawbacks:
 * For most objects, the data is "fragmented" over several tables. For instance, object `HeatPump` is a specialization of object `EnergyConversionSystem`. Therefore table `nrg8_heat_pump` holds only the attributes specific to `HeatPump`, all attributes it shares with `EnergyConversionSystem` are stored in table `nrg8_conv_system`.
 * Some tables are used for more than one type of object. For instance, objects `Cable` and `Canal` are written to the same table `utn9_distrib_element`.

Fortunately, the extended 3DCityDB finds a remedy for this situation: The `citydb_view` schema defines so-called [views](https://en.wikipedia.org/wiki/View_(SQL)) that present the information in a more complete manner. However, the database contains no information about which types are represented in which views, which is why the user has to provide this information.

Therefore, in the following lines the relevant classes are mapped from their respective views.

In [ ]:
SimpleFunctionalElement = access.map_citydb_object_class( 'SimpleFunctionalElement',
    table_name='utn9_ntw_feat_simple_funct_elem', schema='citydb_view'
    )
    
ComplexFunctionalElement = access.map_citydb_object_class( 'ComplexFunctionalElement',
    table_name='utn9_ntw_feat_complex_funct_elem', schema='citydb_view'
    )

TerminalElement = access.map_citydb_object_class( 'TerminalElement',
    table_name='utn9_ntw_feat_term_elem', schema='citydb_view'
    )

Cable = access.map_citydb_object_class( 'Cable',
    table_name='utn9_ntw_feat_distrib_elem_cable', schema='citydb_view'
    )
    
Node = access.map_citydb_object_class( 'Node',
    table_name='utn9_node', schema='citydb_view'
    )

InteriorFeatureLink = access.map_citydb_object_class( 'InteriorFeatureLink',
    table_name='utn9_link_interior_feature', schema='citydb_view'
    )

InterFeatureLink = access.map_citydb_object_class( 'InterFeatureLink',
    table_name='utn9_link_interfeature', schema='citydb_view'
    )

After the mapping, the data itself can be retrieved using function `get_citydb_objects` of class `DBAccess`.

When retrieving the data, it is possible to specify additional selection criteria. For instance, both *busses* and *switches* are stored as `SimpleFunctionalElement`, but with different `class` attributes. Therefore, when retrieving the busses, the additional condition of being either a *busbar*, *plate*, *supply point* or *pole* is defined.

In [ ]:
busses = access.get_citydb_objects( 'SimpleFunctionalElement',
    conditions = [ 
        or_(
            getattr( SimpleFunctionalElement, 'class' ) == 'busbar',
            getattr( SimpleFunctionalElement, 'class' ) == 'plate',
            getattr( SimpleFunctionalElement, 'class' ) == 'supply point',
            getattr( SimpleFunctionalElement, 'class' ) == 'pole'
            )
        ]
    )

lines = access.get_citydb_objects( 'Cable' )

loads = access.get_citydb_objects( 'TerminalElement',
    conditions = [ getattr( TerminalElement, 'class' ) == 'load' ]
    )

transformers = access.get_citydb_objects( 'ComplexFunctionalElement',
    conditions = [ getattr( ComplexFunctionalElement, 'class' ) == 'transformer' ]
    )

switches = access.get_citydb_objects( 'SimpleFunctionalElement',
    conditions = [ getattr( SimpleFunctionalElement, 'class' ) == 'switch' ]
    )

external_grid = access.get_citydb_objects( 'TerminalElement',
    conditions = [ getattr( TerminalElement, 'class' ) == 'external-grid' ]
    )

feature_graphs = access.get_citydb_objects( 'FeatureGraph' )

nodes = access.get_citydb_objects( 'Node' )

interior_feature_links = access.get_citydb_objects( 'InteriorFeatureLink' )

inter_feature_links = access.get_citydb_objects( 'InterFeatureLink' )

## Working with the data
As mentioned above, all data is stored as objects, with their respective data attributes as variables with the corresponding name.

This makes it easy to work with this data. For instance, one can easily compile dictionaries of *network features*, *feature graphs* and *feature graph nodes*:

In [ ]:
# Map network features to their respective IDs.
network_features = { feature.id: feature for feature in ( busses + lines + transformers + switches + external_grid + loads ) }

# Map feature graph IDs to their associated network feature IDs.
feature_ids = { feature_graph.id: feature_graph.ntw_feature_id  for feature_graph in feature_graphs }

# Map feature graph node IDs to the name of the feature graph node.
node_names = { node.id : node.name for node in nodes }

### Exploring the topological data
All components of the power grid model are associated to *network features*, which basically hold the topographical information. These network features are assocaited to *feature graphs*, which hold the topological information. Feature graphs are constructed from *feature graph nodes*. These nodes can be either linked with a feature graphs (*interior links*) or to nodes of other feature graphs (*inter feature links*).

The following code shows how to use this topological information. With the help of package `networkx`, a graph of all nodes and links is created. Different links (interior vs. inter feature links) are shown in different colors. Nodes associated to different types of network features are shown in different colors, too.

**Note**: The figure should be plotted in "interactive mode", making it possible to zoom in. This allows to explore the topological graph in more detail (e.g., types of links connecting different types of nodes).

In [ ]:
import networkx as nx

def get_node_color( feature_graph_id ):
    feature_color_map = {
        'Cable': 'g',
        'SimpleFunctionalElement': 'm',
        'ComplexFunctionalElement': 'r',
        'TerminalElement': 'c'
        }
    feature_id = feature_ids[ feature_graph_id ]
    feature_type = network_features[ feature_id ].classname
    return feature_color_map[ feature_type ]

def get_link_color( link_type ):
    link_color_map = {
        'interior': 'g',
        'inter': 'b'
        }
    return link_color_map[ link_type ]

# Create new networkx graph.
g = nx.Graph()

# Add all feature graph nodes as nodes of the networks graph. Define different
# color attributes depending on what type of network feature they are associated to.
g.add_nodes_from(
    [ ( node_names[node.id], { 'color': get_node_color( node.feat_graph_id ) } ) for node in nodes ]
)

# Add all inter feature links as edges of the networks graph.
g.add_edges_from( 
    [ ( node_names[link.start_node_id], node_names[link.end_node_id] ) for link in interior_feature_links ],
    link_type = 'interior',
    color = get_link_color( 'interior')
    )

# Add all interior links as edges of the networks graph.
g.add_edges_from(
    [ ( node_names[link.start_node_id], node_names[link.end_node_id] ) for link in inter_feature_links ],
    link_type = 'inter', 
    color = get_link_color( 'inter')
    )

# Import matplotlib package.
%matplotlib notebook
import matplotlib.pyplot as plt

# Extract color attributes from nodes and edges.
edge_colors = [ g[u][v]['color'] for u,v in g.edges() ]
node_colors = [ g.nodes[n]['color'] for n in g.nodes() ]

# Draw the graph.
pos = nx.spring_layout( g, scale = 0.5, iterations = 5000 )
nx.draw( g, pos, with_labels=True, node_color=node_colors, edge_color=edge_colors, width=2 )

# In addition, add the link type (inter/interior) as label for all edges.
edge_labels = nx.get_edge_attributes( g, 'link_type' )
nx.draw_networkx_edge_labels( g, pos, edge_labels )

# Show the figure.
plt.draw()

### Exploring the topographical data
As mentioned above, all components of the power grid model are associated to network features, which basically hold the topographical information. This data is for instance useful to calculate line lengths or visualize the network. The latter is demonstrated in the next example.

Note: The figure should be plotted in "interactive mode", making it possible to zoom in. This allows to explore the topographical view in more detail (especially at the origin of the figure).

In [ ]:
%matplotlib notebook
from matplotlib.path import Path
import matplotlib.patches as patches
from pygeoif import from_wkt

fig, ax = plt.subplots()

#
# The following lines extract the geometric data from the various
# types of network features defined in the power grid model.
#

line_plt = []
for line in lines:
    line_geom_wkt = access.execute_function( geom_as_text( line.geom ) )
    line_coords = from_wkt( line_geom_wkt ).coords
    verts = [ ( c[0], c[1] ) for c in line_coords ]
    codes = [ Path.MOVETO, Path.LINETO ]    
    line_plt.append( Path( verts, codes ) )

transformer_plt = []
for transformer in transformers:
    transformer_geom_wkt = access.execute_function( geom_as_text( transformer.geom ) )
    transformer_coords = from_wkt( transformer_geom_wkt ).coords
    verts = [ ( c[0], c[1] ) for c in transformer_coords ]
    codes = [ Path.MOVETO, Path.LINETO ]    
    transformer_plt.append( Path( verts, codes ) )

switch_plt = []
for switch in switches:
    switch_geom_wkt = access.execute_function( geom_as_text( switch.geom ) )
    switch_coords = from_wkt( switch_geom_wkt ).coords
    verts = [ ( c[0], c[1] ) for c in switch_coords ]
    codes = [ Path.MOVETO, Path.LINETO ]    
    switch_plt.append( Path( verts, codes ) )

bus_plt = []
for bus in busses:
    bus_geom_wkt = access.execute_function( geom_as_text( bus.geom ) )
    bus_coords = from_wkt( bus_geom_wkt ).coords
    bus_plt.append( [ bus_coords[0][0], bus_coords[0][1] ] )

load_plt = []
for load in loads:
    load_geom_wkt = access.execute_function( geom_as_text( load.geom ) )
    load_coords = from_wkt( load_geom_wkt ).coords
    load_plt.append( [ load_coords[0][0], load_coords[0][1] ] )

#
# The remaining lines call matplotlib and add all elements to the plot.
#

for path in line_plt:
    patch = patches.PathPatch( path, edgecolor='black', lw=2 )
    ax.add_patch( patch )

for path in transformer_plt:
    patch = patches.PathPatch( path, edgecolor='orange', lw=2 )
    ax.add_patch( patch )

for path in switch_plt:
    patch = patches.PathPatch( path, edgecolor='blue', lw=2 )
    ax.add_patch( patch )

plt.plot( *zip(*bus_plt), marker='o', color='r', ls='')
plt.plot( *zip(*load_plt), marker='o',  markersize=8, color='g', ls='')

ax.set_xlim(-250,330)
ax.set_ylim(-120,220)

plt.show()

That's all, folks!